In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQLExample")\
        .master("spark://spark-master:7077").getOrCreate() 
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = spark.read.format("csv")\
          .option("header", "true")\
          .option("inferSchema", "true")\
          .load("/home/jovyan/data/online-retail-dataset.csv")\
          .coalesce(5)
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

Register the dataframe as a relational table

In [43]:
df.createOrReplaceTempView("dfTable") # Register the dataframe as a relational table

In [44]:
spark.sql("SELECT COUNT(*) FROM dfTable").show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [45]:
spark.sql("SELECT approx_count_distinct(StockCode, 0.1) FROM dfTable").show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [46]:
spark.sql("SELECT min(Quantity), max(Quantity), sum(Quantity) FROM dfTable").show()

+-------------+-------------+-------------+
|min(Quantity)|max(Quantity)|sum(Quantity)|
+-------------+-------------+-------------+
|       -80995|        80995|      5176450|
+-------------+-------------+-------------+



In [47]:
spark.sql("SELECT count(*), InvoiceNo, CustomerId FROM dfTable GROUP BY InvoiceNo, CustomerId").show()

+--------+---------+----------+
|count(1)|InvoiceNo|CustomerId|
+--------+---------+----------+
|      32|   563017|     13198|
|      71|   563214|     16370|
|       4|   563372|     15653|
|      31|   563714|     14565|
|       9|   564666|     12492|
|       1|  C565313|     14527|
|      23|   565318|     12921|
|      11|  C565962|     14410|
|      18|   566023|     12955|
|      37|   566079|     17593|
|      19|   566904|     15660|
|      32|   567476|     14859|
|       1|  C567643|     12409|
|       2|   568330|     16468|
|       4|   568509|     16422|
|       1|  C568911|     15050|
|      49|   569105|     16729|
|      25|   569211|     15774|
|       1|   569388|     14031|
|      54|   570179|     17509|
+--------+---------+----------+
only showing top 20 rows



In [51]:
from pyspark.sql.functions import col, to_date
df.show(10)
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.printSchema()
dfNoNull = dfWithDate.dropna()
dfWithDate.show(10)
dfNoNull.createOrReplaceTempView("dfNoNull")
spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode)) ORDER BY CustomerId DESC, stockCode DESC").show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [53]:
# Stop the Spark context
spark.stop()